In [1]:
import time
time.strftime("%c")

'Thu Sep 29 15:56:57 2016'

In [2]:
import os

from collections import defaultdict
from itertools import product

import pandas as pd
import requests

In [3]:
base = os.environ['BUG_FREE_EUREKA_BASE']
roster_path = os.path.join(base, 'grading', 'roster.tsv')

In [4]:
assignment_1 = ['DNA', 'RNA', 'GC', 'HAMM', 'REVC']
assignment_2 = ['INI', 'DBPR', 'FIB', 'IPRB']
assignment_3 = ['FRMT', 'PROT', 'SUBS', 'CONS', 'FIBD', 'GRPH', 'IEV', 'MPRT', 'MRNA', 'PERM', 'PRTM']
assignment_all = assignment_1 + assignment_2 + assignment_3

In [ ]:
roster = pd.read_csv(roster_path, index_col=0, sep='\t')

# Checking GitHub

In [6]:
gh_check = requests.get('https://api.github.com/rate_limit')
gh_check_json = gh_check.json()

if gh_check_json['resources']['core']['remaining'] < len(roster.index):
    raise Exception('Not enough GitHub API calls left. Wait {}'.format(gh_check_json['resources']['core']['reset']))

In [7]:
gh_fmt = 'https://api.github.com/repos/{}/BioDb2016/contents/'

gh_result_dict = defaultdict(list)

for name, github_user, rosalind_user in roster.itertuples():
    print('Checking GitHub: {} ({})'.format(name, github_user))
    
    response = requests.get(gh_fmt.format(github_user))
    
    for d in response.json():
        if 'type' not in d:
            raise Exception('Missing data.')
        if 'file' == d['type']:
            gh_result_dict[name].append(d['name'])

Checking GitHub: Akrishta (akrishta305)
Checking GitHub: Alex (alexmasny)
Checking GitHub: Apurva (ApurvaG05)
Checking GitHub: Arunabh (arunabh99)
Checking GitHub: Asma (AsmaTahir)
Checking GitHub: Xin (huxin0221)
Checking GitHub: Ihsan (ihsanmuchsin92)
Checking GitHub: Amir (kayvanjoo)
Checking GitHub: Lilit (lilith3)
Checking GitHub: Ming (MingBit)
Checking GitHub: Monica (monicaSudarsanam)
Checking GitHub: Oyelle (oyelle)
Checking GitHub: Randa (randaelsh)
Checking GitHub: Sabby (sabyasachi2k13)
Checking GitHub: Sakthi (Sakthi23)
Checking GitHub: Sepehr (sepehrgolriz)
Checking GitHub: Tamal (tamalarafat)
Checking GitHub: Umesh (thapaumesh)
Checking GitHub: Haritha (thummagunta)
Checking GitHub: Hanan (wareef)
Checking GitHub: Nanditha (NandithaMallesh)


In [8]:
gh_results = defaultdict(dict)

for name, files in gh_result_dict.items():
    filenames = [file.split('.')[0] for file in files]
    for assignment in assignment_all:
        gh_results[name][assignment] = 1 if assignment in filenames else 0
    
gh_result_df = pd.DataFrame.from_dict(gh_results, orient='index')

# Checking Rosalind

In [9]:
rosalind_result_dict = defaultdict(dict)

for name, assignment in product(roster.index, assignment_all):
    rosalind_result_dict[name][assignment] = 0

In [10]:
for name, github, rosalind in roster.itertuples():
    print('Checking Rosalind: {} ({})'.format(name, rosalind))

    res = requests.get(rosalind)

    for line in res.iter_lines():
        line = line.decode('utf-8').strip()
        if 'badge-success' not in line:
            continue
        success = line[1 + line.find('>'): line.find('<', 2)]
        rosalind_result_dict[name][success] = 1

Checking Rosalind: Akrishta (http://rosalind.info/users/akrishta/)
Checking Rosalind: Alex (http://rosalind.info/users/alexmasny/)
Checking Rosalind: Apurva (http://rosalind.info/users/ApurvaG05/)
Checking Rosalind: Arunabh (http://rosalind.info/users/arunabh99/)
Checking Rosalind: Asma (http://rosalind.info/users/AsmaTahir/)
Checking Rosalind: Xin (http://rosalind.info/users/huxin0221/)
Checking Rosalind: Ihsan (http://rosalind.info/users/ihsanm/)
Checking Rosalind: Amir (http://rosalind.info/users/kayvanjo/)
Checking Rosalind: Lilit (http://rosalind.info/users/lilit.antonyan/)
Checking Rosalind: Ming (http://rosalind.info/users/MingBit/)
Checking Rosalind: Monica (http://rosalind.info/users/monicaSudarsanam/)
Checking Rosalind: Oyelle (http://rosalind.info/users/nazneen/)
Checking Rosalind: Randa (http://rosalind.info/users/randaelsh/)
Checking Rosalind: Sabby (http://rosalind.info/users/sabyasachi2k13/)
Checking Rosalind: Sakthi (http://rosalind.info/users/Sakthi23/)
Checking Rosali

In [11]:
rosalind_result_df = pd.DataFrame(rosalind_result_dict).T
rosalind_result_df[rosalind_result_df.isnull()] = 0 # sanitize from missing assignments
rosalind_result_df = rosalind_result_df.astype(int)

# Summary

In [12]:
print('Assignment 1 has {} tasks'.format(len(assignment_1)))

pd.DataFrame({
    'GitHub': gh_result_df[assignment_1].sum(axis=1),
    'Rosalind': rosalind_result_df[assignment_1].sum(axis=1)
})

Assignment 1 has 5 tasks


,GitHub,Rosalind
Akrishta,0,1
Alex,5,5
Amir,5,1
Apurva,3,3
Arunabh,4,5
Asma,1,4
Hanan,5,0
Haritha,0,0
Ihsan,5,5
Lilit,0,4


In [13]:
print('Assignment 2 has {} tasks'.format(len(assignment_2)))

pd.DataFrame({
    'GitHub': gh_result_df[assignment_2].sum(axis=1),
    'Rosalind': rosalind_result_df[assignment_2].sum(axis=1)
})

Assignment 2 has 4 tasks


,GitHub,Rosalind
Akrishta,0,0
Alex,0,0
Amir,0,0
Apurva,0,0
Arunabh,0,0
Asma,0,0
Hanan,0,0
Haritha,0,0
Ihsan,0,1
Lilit,0,0


In [14]:
print('Assignment 3 has {} tasks'.format(len(assignment_3)))

pd.DataFrame({
    'GitHub': gh_result_df[assignment_3].sum(axis=1),
    'Rosalind': rosalind_result_df[assignment_3].sum(axis=1)
})

Assignment 3 has 11 tasks


,GitHub,Rosalind
Akrishta,0,0
Alex,2,2
Amir,0,0
Apurva,0,0
Arunabh,0,0
Asma,0,0
Hanan,0,0
Haritha,0,0
Ihsan,0,0
Lilit,0,0
